In [53]:
import numpy as np
from numpy import linalg
import functools as ft
from operator import add
import os

Функция генерации строки для np.einsum

In [54]:
def generate_index(tensor_len, n):
    z_array = np.array(["z"] * tensor_len)
    
    res = list(map(chr, np.arange(ord('a'), ord('a') + tensor_len, 1, dtype=int)))
    tmp_2 = list(map(add, res, z_array))
    
    res = ''.join(res)
    
    for i in range(tensor_len - 1, -1, -1):
        if i != n:
            res += "," + tmp_2[i]
        
    return res + "->" + res[n] + "z"

In [55]:
def main():
    R = 3 #rank
    iterations = 6 #number of iterations
    
    T = np.random.randn(2, 3, 4, 5) #random tensor
    
    U = [np.random.randn(_, R) for _ in np.shape(T)] #factor matrices -> computed CP
    
    H = [0] * len(np.shape(T))
    M = [0] * len(np.shape(T))

    for k in range(iterations):
        for n in range(len(np.shape(T))):
            U_copy = U.copy()
            U_copy.pop(n)

            M[n] = np.einsum(generate_index(len(np.shape(T)), n), T, *U_copy[::-1])
            
            UtU = [U_copy[i].T @ U_copy[i] for i in range(len(U_copy))]

            H[n] = ft.reduce(np.multiply, UtU)
            
            U[n] = M[n] @ np.linalg.pinv(H[n])
        
        
    for i in range(len(np.shape(T))):
        print(U[i])

In [56]:
if __name__ == "__main__":
    main()

[[ 0.07622976  0.21731574  0.09919197]
 [-0.90907069  1.23305801 -0.02126228]]
[[-0.02910252  1.71626268 -1.40832341]
 [-0.50222966  1.97891602  0.10416468]
 [-0.38277864  0.60025987  0.33479017]]
[[ 3.51904116 -0.83072345  6.12427194]
 [-0.46986956  0.46388282  3.90047035]
 [-2.38780767 -0.15689109 -4.00050835]
 [-0.93589707  0.69580911 -3.25115521]]
[[-0.98796606 -1.00916529  2.68143983]
 [ 1.05275112  0.36221344 -0.89253818]
 [ 0.56214104  0.48989523  1.70060205]
 [ 1.3704857   0.37545414  0.93487213]
 [-1.18943955 -0.64824348 -0.63182333]]
